In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from Agent import ChargingAgentAll, ChargingAgentIndividual, reset_params_file, clear_param_backups
from flaml.autogen.code_utils import extract_code

In [2]:
# global params_filepath
params_filepath = "Model/params/EVCharging.json"
params_filepath_backup = "Model/params/EVCharging_original.json"

In [3]:
reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [4]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

In [5]:
config_list_gpt3_5 = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-16k"],
    },
)

In [6]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

In [7]:
code_path = "Model/EVCharging.py"

In [8]:
with open(code_path) as f:
    code = f.read()

In [9]:
example_qa = """
----------
Instruction: Process this request for parking lot manager
Change carbon cost to 30
Answer Code:
```JSON
"lot_params":{"carbon_cost": 30}
```

----------
Instruction: Process this request for car ZNV0672
Update end time to 10 PM
Answer Code:
```JSON
"ZNV0672":{"end_time": 22}
```
"""

In [10]:
agentAll = ChargingAgentAll(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x2ca4d787
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -135.7200000
Presolve removed 56 rows and 192 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -142.04 -135.72 
No other solutions better than -142.04

Optimal solution found (tolerance 1.00e-04)
Best objective -1.420400000000e+02, best bound -1.420400000000e+02, gap 0.0

In [11]:
agentIndividual = ChargingAgentIndividual(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x2ca4d787
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -135.7200000
Presolve removed 56 rows and 192 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -142.04 -135.72 
No other solutions better than -142.04

Optimal solution found (tolerance 1.00e-04)
Best objective -1.420400000000e+02, best bound -1.420400000000e+02, gap 0.0000%
got status


In [12]:
user_manager = UserProxyAgent(
    "manager", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

user_car1 = UserProxyAgent(
    "HVC0538", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

user_car2 = UserProxyAgent(
    "ZNV0672", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

# Simple requests

## Manager

In [13]:
user_manager.initiate_chat(agentAll, message="Change energy cost weight to 1", clear_history=True)

manager (to Tesla Charging Example):

Change energy cost weight to 1

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"lot_params":{"elec_cost_weight": 1}
```

--------------------------------------------------------------------------------
{'lot_params': {'elec_cost_weight': 1}}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x488cb761
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -124.200

In [14]:
list(log_history.values())[0]['cost']

[0.02898]

In [15]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [16]:
user_manager.initiate_chat(agentIndividual, message="Change energy cost weight to 1", clear_history=True)

manager (to Tesla Charging Example):

Change energy cost weight to 1

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"elec_cost_weight": 1
```

--------------------------------------------------------------------------------
{'elec_cost_weight': 1}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x488cb761
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -124.2000000
Presolve removed 56 rows a

In [17]:
list(log_history.values())[0]['cost']

[0.01635]

In [18]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

## Car Owner

In [19]:
user_car1.initiate_chat(agentAll, message="Update end time to 7 PM", clear_history=True)

HVC0538 (to Tesla Charging Example):

Update end time to 7 PM

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"HVC0538":{"end_time": 19}
```

--------------------------------------------------------------------------------
{'HVC0538': {'end_time': 19}}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0xbf4f3ad1
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -121.8000000
Presolve removed 56 ro

In [20]:
list(log_history.values())[0]['cost']

[0.028919999999999998]

In [21]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [22]:
user_car1.initiate_chat(agentIndividual, message="Update end time to 7 PM", clear_history=True)

HVC0538 (to Tesla Charging Example):

Update end time to 7 PM

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{"end_time": 19}
```

--------------------------------------------------------------------------------
{'end_time': 19}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0xbf4f3ad1
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -121.8000000
Presolve removed 56 rows and 192 columns
Pres

In [23]:
list(log_history.values())[0]['cost']

[0.009300000000000001]

In [24]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

# Complex Requests

In [25]:
user_manager.initiate_chat(agentAll, message="Change energy cost weight to 1 and increase electricity cost by 20%", clear_history=True)

manager (to Tesla Charging Example):

Change energy cost weight to 1 and increase electricity cost by 20%

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"lot_params":{
    "elec_cost_weight": 1,
    "elec_cost": {
        "0": 0.36,
        "1": 0.36,
        "2": 0.36,
        "3": 0.36,
        "4": 0.36,
        "5": 0.36,
        "6": 0.36,
        "7": 0.42,
        "8": 0.42,
        "9": 0.42,
        "10": 0.42,
        "11": 0.36,
        "12": 0.36,
        "13": 0.36,
        "14": 0.36,
        "15": 0.36,
        "16": 0.36,
        "17": 0.36,
        "18": 0.36,
        "19": 0.42,
        "20": 0.42,
        "21": 0.42,
        "22": 0.36,
        "23": 0.36
    }
}
```

--------------------------------------------------------------------------------
{'lot_par

In [26]:
list(log_history.values())[0]['cost']

[0.042809999999999994]

In [27]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [28]:
user_manager.initiate_chat(agentIndividual, message="Change energy cost weight to 1 and increase electricity cost by 20%", clear_history=True)

manager (to Tesla Charging Example):

Change energy cost weight to 1 and increase electricity cost by 20%

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
  "elec_cost_weight": 1,
  "elec_cost": {"0": 0.36, "1": 0.36, "2": 0.36, "3": 0.36, "4": 0.36, "5": 0.36, "6": 0.36, "7": 0.42, "8": 0.42, "9": 0.42, "10": 0.42, "11": 0.36, "12": 0.36, "13": 0.36, "14": 0.36, "15": 0.36, "16": 0.36, "17": 0.36, "18": 0.36, "19": 0.42, "20": 0.42, "21": 0.42, "22": 0.36, "23": 0.36}
}
```

--------------------------------------------------------------------------------
{'elec_cost_weight': 1, 'elec_cost': {'0': 0.36, '1': 0.36, '2': 0.36, '3': 0.36, '4': 0.36, '5': 0.36, '6': 0.36, '7': 0.42, '8': 0.42, '9': 0.42, '10': 0.42, '11': 0.36, '12': 0.36, '13': 0.36, '14': 0.36, '15': 0.36, '16'

In [30]:
print(list(log_history.values())[0]['cost'][0])

log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [33]:
user_car1.initiate_chat(agentAll, message="Update end time to 7 PM and required charge to 20", clear_history=True)

HVC0538 (to Tesla Charging Example):

Update end time to 7 PM and required charge to 20

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"HVC0538": {
    "end_time": 19,
    "required_charge": 20
}
```

--------------------------------------------------------------------------------
{'HVC0538': {'end_time': 19, 'required_charge': 20}}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0xaba67db0
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-02, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e

In [34]:
print(list(log_history.values())[0]['cost'][0])

log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

0.02982


In [35]:
user_car1.initiate_chat(agentIndividual, message="Update end time to 7 PM and required charge to 20", clear_history=True)

HVC0538 (to Tesla Charging Example):

Update end time to 7 PM and required charge to 20

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
"end_time": 19,
"required_charge": 20
}
```

--------------------------------------------------------------------------------
{'end_time': 19, 'required_charge': 20}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0xaba67db0
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-02, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solu

In [36]:
print(list(log_history.values())[0]['cost'][0])

log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

0.01002
